# 🧠 Notebook: Chuyển dữ liệu Vinmec sang định dạng ViMQ
Notebook này giúp bạn tự động sinh tập dữ liệu ViMQ từ file CSV gốc `Vinmec_diseases_information.csv`.

In [1]:
%pip install pandas -q


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Bước 1: Đọc dữ liệu từ CSV
import pandas as pd

file_path = 'Vinmec_diseases_information.csv'
df = pd.read_csv(file_path)
df = df.dropna(subset=['name'])
df.head()

,name,description,causes,symptoms,contagion,risk_subjects,prevention,diagnosis,treatment
0,Áp xe,Áp xe là gì? Áp xe là tên gọi của một tổ chức ...,Nhiễm trùng là nguyên nhân trực tiếp phổ biến ...,"Biểu hiện lâm sàng của áp xe khá đặc hiệu, bao...",Áp xe có thể được lây truyền do tác nhân gây b...,Những người có các đặc điểm sau có khả năng hì...,Áp xe có thể được phòng ngừa bằng các biện phá...,Chẩn đoán áp xe mô dưới da thường được thực hi...,Một tổ chức áp xe nếu không được điều trị sẽ t...
1,Áp xe vú,"Áp xe vú là tình trạng viêm, sưng đỏ, có hạch ...",Nguyên nhân áp xe vú thường gặp nhất là do hai...,Dấu hiệu áp xe vú phụ thuộc vào vị trí bị áp x...,no_information,Có từ 10-30% phụ nữ sau khi mang thai và cho c...,Biện pháp để phòng ngừa áp xe vú cần chú ý: Gi...,Chẩn đoán áp xe vú thông qua các dấu hiệu áp x...,Điều trị áp xe vú có thể sử dụng thuốc và chíc...
2,Áp xe hậu môn,Áp xe hậu môn là tình trạng mưng mủ ở gần khu ...,Có rất nhiều nguyên nhân dẫn đến áp xe hậu môn...,Triệu chứng phổ biến nhất của áp-xe hậu môn là...,no_information,no_information,"Nếu được điều trị và chăm sóc phù hợp, người b...",Chẩn đoán áp xe hậu môn thông thường nhất kiểm...,"Khi khối mủ vùng hậu môn vỡ, người bệnh cần đư..."
3,Áp xe phổi,Bệnh áp xe phổi là bệnh lý nhiễm trùng tại phổ...,Nguyên nhân gây bệnh áp xe phổi có thể chia th...,Triệu chứng lâm sàng của bệnh áp xe phổi thườn...,Áp xe phổi có lây không? Áp xe phổi có thể lây...,Các yếu tố làm tăng nguy cơ mắc bệnh áp xe phổ...,Các biện pháp giúp phòng ngừa và hạn chế diễn ...,Việc chẩn đoán áp xe phổi cần phối hợp giữa cá...,Điều trị bệnh áp xe phổi cần sự phối hợp của n...
4,Alzheimer,Bệnh Alzheimer là một trong những căn bệnh về ...,Hiện nay vẫn chưa xác định nguyên nhân gây ra ...,"Bệnh alzheimer có các dấu hiệu, triệu chứng bệ...",no_information,"Là một căn bệnh khá phổ biến, bệnh alzheimer t...","Để phòng ngừa bệnh Alzheimer, có thể áp dụng c...",Để chuẩn đoán bệnh nhân có bị bệnh Alzheimer k...,"Hiện nay, vẫn chưa có phương pháp để điều trị ..."


In [3]:
# Bước 2: Thiết lập các mẫu câu hỏi và nhãn loại câu hỏi
question_templates = {
    'description': ('Bệnh {name} là gì?', 'definition'),
    'symptoms': ('Triệu chứng của bệnh {name} là gì?', 'symptoms'),
    'causes': ('Nguyên nhân gây ra bệnh {name} là gì?', 'cause'),
    'contagion': ('Bệnh {name} có lây không?', 'contagion'),
    'risk_subjects': ('Ai có nguy cơ mắc bệnh {name}?', 'risk'),
    'prevention': ('Làm sao để phòng bệnh {name}?', 'prevention')
}

In [4]:
# Bước 3: Sinh tập dữ liệu theo định dạng ViMQ
viMQ_data = []

for _, row in df.iterrows():
    name = row['name']
    name_tokens = name.strip().split()

    for col, (template, sent_label) in question_templates.items():
        if col in row and isinstance(row[col], str) and row[col].strip().lower() != 'no_information':
            sentence = template.format(name=name).strip()
            tokens = sentence.split()
            try:
                start_idx = tokens.index(name_tokens[0])
                end_idx = start_idx + len(name_tokens) - 1
                viMQ_data.append({
                    'sentence': sentence,
                    'seq_label': [[start_idx, end_idx, 'SYMPTOM_AND_DISEASE']],
                    'sent_label': sent_label
                })
            except ValueError:
                continue

print(f'✅ Sinh được {len(viMQ_data)} dòng dữ liệu hỏi đáp.')

✅ Sinh được 3497 dòng dữ liệu hỏi đáp.


In [5]:
# Bước 4: Xuất ra file JSON
import json

output_path = 'vinmec_vimq_full.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(viMQ_data, f, ensure_ascii=False, indent=2)

print(f'✅ Đã lưu file: {output_path}')

✅ Đã lưu file: vinmec_vimq_full.json
